In [22]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [23]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [24]:
tool = TavilySearchResults(max_results=4)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [25]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [26]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        
        # Returns a runnable graph
        self.graph = graph.compile()
        # Let model know about tools
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [27]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [28]:
# This did not work as there was issue with installing pugraphviz
#from IPython.display import Image
#Image(abot.graph.get_graph().draw_png())

In [29]:
messages = [HumanMessage(content="What is the weather in Zurich?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Zurich'}, 'id': 'call_QklCTj52qRxD6PGGoaCzvGdG', 'type': 'tool_call'}
Back to the model!


In [30]:
print(result['messages'][-1].content)


The current weather in Zurich is partly cloudy with a temperature of 20.3°C (68.5°F). The wind speed is 3.4 mph from the west direction. The humidity is at 40%, and the visibility is 10.0 km (6.0 miles).


In [19]:
messages = [HumanMessage(content="What is the weather in Zurich and Berlin?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Zurich'}, 'id': 'call_xefqQVqpSZxWecFHhpvhxxYB', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Berlin'}, 'id': 'call_1LhZUxLFtAnPjexfcZgSJpTc', 'type': 'tool_call'}
Back to the model!


In [20]:
print(result['messages'][-1].content)

The current weather in Zurich is sunny with a temperature of 20.3°C (68.5°F) and a wind speed of 7.2 km/h coming from the west. The humidity is at 35%, and there is no precipitation at the moment. In Berlin, the weather is partly cloudy with a temperature of 14.3°C (57.7°F). The wind speed is 5.4 km/h from the west, and the humidity is at 63%.


In [21]:
model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)

query = "Who won the eurocup in 2024? What is the GDP of that country? Answer each question." 
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Euro Cup 2024 winner'}, 'id': 'call_4enqdSGUMgjxeucn9rFqBk1k', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Spain GDP 2024'}, 'id': 'call_gJTn0zsNkSjw6afvqbK4fdGi', 'type': 'tool_call'}
Back to the model!
Spain won the Euro Cup in 2024. The GDP of Spain in 2024 was approximately €1.593 trillion ($1.724 trillion), with a growth rate of 3.2% compared to the previous year.
